##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrar SessionRunHook a retrollamadas Keras

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/sessionrunhook_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>


En TensorFlow 1, para personalizar el comportamiento del entrenamiento, usas `tf.estimator.SessionRunHook` con `tf.estimator.Estimator`. Esta guía muestra cómo migrar de `SessionRunHook` a las retrollamadas personalizadas de TensorFlow 2 con la API `tf.keras.callbacks.Callback`, que funciona con `Model.fit` de Keras para el entrenamiento (así como `Model.evaluate` y `Model.predict`). Aprenderás a hacerlo implementando un `SessionRunHook` y una tarea `Callback` que mida los ejemplos por segundo durante el entrenamiento.

Ejemplos de retrollamadas son el guardado de puntos de verificación (`tf.keras.callbacks.ModelCheckpoint`) y la escritura de sumarios [TensorBoard](%60tf.keras.callbacks.TensorBoard%60). Las [retrollamadas](https://www.tensorflow.org/guide/keras/custom_callback) de Keras son objetos a los que se llama en distintos momentos del entrenamiento/evaluación/predicción en las APIs `Model.fit`/`Model.evaluate`/`Model.predict` incorporadas de Keras. Puedes obtener más información sobre las retrollamadas en la documentación de la API `tf.keras.callbacks.Callback` así como en las guías [Cómo escribir tus propias retrollamadas](https://www.tensorflow.org/guide/keras/custom_callback.ipynb/) y [Entrenamiento y evaluación con los métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (sección *Usar retrollamadas*).

## Preparación

Empieza con imports y un conjunto de datos sencillo a modo de demostración:

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

import time
from datetime import datetime
from absl import flags

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: Crear un SessionRunHook personalizado con las API tf.estimator

Los siguientes ejemplos de TensorFlow 1 muestran cómo configurar un `SessionRunHook` personalizado que mida los ejemplos por segundo durante el entrenamiento. Tras crear el hook (`LoggerHook`), pásalo al parámetro `hooks` de `tf.estimator.Estimator.train`.

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (features, labels)).batch(1).repeat(100)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [ ]:
class LoggerHook(tf1.train.SessionRunHook):
  """Logs loss and runtime."""

  def begin(self):
    self._step = -1
    self._start_time = time.time()
    self.log_frequency = 10

  def before_run(self, run_context):
    self._step += 1

  def after_run(self, run_context, run_values):
    if self._step % self.log_frequency == 0:
      current_time = time.time()
      duration = current_time - self._start_time
      self._start_time = current_time
      examples_per_sec = self.log_frequency / duration
      print('Time:', datetime.now(), ', Step #:', self._step,
            ', Examples per second:', examples_per_sec)

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
estimator.train(_input_fn, hooks=[LoggerHook()])

## TensorFlow 2: Crear una retrollamada Keras personalizada para Model.fit

En TensorFlow 2, cuando usas el Keras incorporado `Model.fit` (o `Model.evaluate`) para entrenamiento/evaluación, puedes configurar una `tf.keras.callbacks.Callback` personalizada, que luego pasas al parámetro `callbacks` de `Model.fit` (o `Model.evaluate`). (Obtén más información en la guía [Cómo escribir tus propias retrollamadas](../..guide/keras/custom_callback.ipynb)).

En el ejemplo siguiente, escribirás una `tf.keras.callbacks.Callback` personalizada que registre varias métricas: medirá los ejemplos por segundo, que deberían ser comparables a las métricas del ejemplo anterior `SessionRunHook`.

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs = None):
      self._step = -1
      self._start_time = time.time()
      self.log_frequency = 10

    def on_train_batch_begin(self, batch, logs = None):
      self._step += 1

    def on_train_batch_end(self, batch, logs = None):
      if self._step % self.log_frequency == 0:
        current_time = time.time()
        duration = current_time - self._start_time
        self._start_time = current_time
        examples_per_sec = self.log_frequency / duration
        print('Time:', datetime.now(), ', Step #:', self._step,
              ', Examples per second:', examples_per_sec)

callback = CustomCallback()

dataset = tf.data.Dataset.from_tensor_slices(
    (features, labels)).batch(1).repeat(100)

model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer, "mse")

# Begin training.
result = model.fit(dataset, callbacks=[callback], verbose = 0)
# Provide the results of training metrics.
result.history

## Siguientes pasos

Aprende más sobre las retrollamadas en:

- Documentación de la API: `tf.keras.callbacks.Callback`
- Guía: [Cómo escribir tus propias retrollamadas](../..guide/keras/custom_callback.ipynb/)
- Guía: [Entrenamiento y evaluación con los métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (la sección *Usar retrollamadas*)

También te pueden resultar útiles los siguientes recursos relacionados con la migración:

- La [Guía de migración de la parada temprana](early_stopping.ipynb): `tf.keras.callbacks.EarlyStopping` es una retrollamada de parada temprana incorporada
- La [Guía de migración a TensorBoard](tensorboard.ipynb): TensorBoard permite el seguimiento y la visualización de métricas
- La [guía de migración de LoggingTensorHook y StopAtStepHook a las retrollamadas de Keras](logging_stop_hook.ipynb)